<a href="https://colab.research.google.com/github/samarkinav/ds_project/blob/main/DSproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2
!pip install natasha

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 8.2MB 6.8MB/s 


In [3]:
import nltk
import pymorphy2
import pandas as pd

In [4]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC' -O Mosru_dump_all_pages.txt

--2021-04-19 08:04:38--  https://docs.google.com/uc?export=download&id=1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC
Resolving docs.google.com (docs.google.com)... 173.194.213.101, 173.194.213.138, 173.194.213.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.213.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7sov580k7b7jfu3srsajqh1lojh4lrrq/1618819425000/03619448018617981983/*/1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC?e=download [following]
--2021-04-19 08:04:40--  https://doc-10-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7sov580k7b7jfu3srsajqh1lojh4lrrq/1618819425000/03619448018617981983/*/1d5NQuYPlkvj2IuHMJ2DFvQ_nXX4f70ZC?e=download
Resolving doc-10-ak-docs.googleusercontent.com (doc-10-ak-docs.googleusercontent.com)... 172.217.193.132, 2607:f8b0:400c:c03::84
Connecting to doc-10-ak-docs.googleusercontent.com (doc-10

In [5]:
df = pd.read_csv('Mosru_dump_all_pages.txt', engine='python', sep="', '", names=['тема','текст','отвественный','пост','дата'])
df.head()

,тема,текст,отвественный,пост,дата
0,'Сообщение о проблеме «Несоблюдение требований...,Выше второго этажа,**********,начальник Объединения административно-техниче...,16 Ноября 2020 в 16:31'
1,'Сообщение о проблеме «Неубранная городская те...,По адресу Снежная д24 расположена музыкальная ...,**********,глава управы района Свиблово города Москвы,05 Мая 2020 в 15:45'
2,'Сообщение о проблеме «Неубранная городская те...,После проведения работ на кабельной канализаци...,**********,начальник Объединения административно-техниче...,01 Марта 2020 в 16:57'
3,'Сообщение о проблеме «Неубранная городская те...,Очистите опору освещения. Приведите в надлежащ...,**********,глава управы района Свиблово города Москвы,16 Января 2020 в 13:54'
4,'Сообщение о проблеме «Захламление территории»...,Более двух недель лежит куча грунта в перемешк...,**************,и.о. главы управы района Свиблово города Москвы,24 Августа 2019 в 15:39'


Обрабатывать даты будем с помощью библиотеки Natasha

In [8]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    NamesExtractor,
    DatesExtractor, 
    MoneyExtractor, 
    AddrExtractor,

    Doc)

segmenter = Segmenter()
morph_vocab = MorphVocab()

# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)
# syntax_parser = NewsSyntaxParser(emb)
# ner_tagger = NewsNERTagger(emb)

dates_extractor = DatesExtractor(morph_vocab)

In [ ]:
def preparing_dates(string):
    matches = dates_extractor(string)
    facts = [_.fact.as_json for _ in matches]

    for f in facts:
        date = (f"{f.get('day')}.{f.get('month')}.{f.get('year')}")
    return date


In [56]:
dff = df.head(20)
test_df = dff.copy()

Обработка даты

In [19]:
for i in range(test_df.shape[0]):
    matches = dates_extractor(test_df.iloc[i, 4])
    facts = [_.fact.as_json for _ in matches]
    
    for f in facts:
        test_df.iloc[i, 4]=(f"{f.get('day')}.{f.get('month')}.{f.get('year')}")

Обработка адреса

In [57]:
import re
address_list = []
solution_list = []
for i in range(test_df.shape[0]):
    matches = dates_extractor(test_df.iloc[i, 4])
    facts = [_.fact.as_json for _ in matches]
    
    for f in facts:
        test_df.iloc[i, 4]=(f"{f.get('day')}.{f.get('month')}.{f.get('year')}")
        
    address_list.append(re.search('по адресу (.*)', test_df.iloc[i, 0]).group(1))
    solution_list.append(re.search('«(.*)»', test_df.iloc[i, 0], flags=re.IGNORECASE).group(1))
test_df['адрес']=address_list
test_df['жалоба']=solution_list
del test_df['тема']




In [58]:
test_df

,текст,отвественный,пост,дата,адрес,жалоба
0,Выше второго этажа,**********,начальник Объединения административно-техниче...,16.11.2020,Снежная улица д.26,Несоблюдение требований к размещению информаци...
1,По адресу Снежная д24 расположена музыкальная ...,**********,глава управы района Свиблово города Москвы,5.5.2020,Снежная улица д.24Д,Неубранная городская территория
2,После проведения работ на кабельной канализаци...,**********,начальник Объединения административно-техниче...,1.3.2020,Снежная улица д.6,Неубранная городская территория
3,Очистите опору освещения. Приведите в надлежащ...,**********,глава управы района Свиблово города Москвы,16.1.2020,Снежная улица д.24Д,Неубранная городская территория
4,Более двух недель лежит куча грунта в перемешк...,**************,и.о. главы управы района Свиблово города Москвы,24.8.2019,Снежная улица д.1А,Захламление территории
5,"Осколки бордюрного камня спрятаны за дерево, в...",************,глава управы района Свиблово города Москвы,5.5.2019,Снежная улица д.12 с.2,Захламление территории
6,Прошу рассмотреть законность размещения вывеск...,************,глава управы района Свиблово города Москвы,6.2.2019,Снежная улица д.26,Несоблюдение требований к размещению информаци...
7,Просьба проверить законность размещения информ...,************,глава управы района Свиблово города Москвы,5.10.2018,Снежная улица д.26,Несоблюдение требований к размещению информаци...
8,Здравствуйте! Прошу проверить законность устан...,************,глава управы района Свиблово города Москвы,22.3.2018,Снежная улица д.23,Несоблюдение требований к размещению информаци...
9,Отсутствуют информационные щиты на всех подобн...,************,генеральный директор Государственного бюджетн...,5.9.2017,Снежная улица д.6,Отсутствие информационного щита
